<a href="https://colab.research.google.com/github/AriyantGit/Movie_Recomendation/blob/main/MOVIE_RECOMMENDATION_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
print("import required packages")

import required packages


In [7]:
#DataSet Import from Github link
movie_url="https://raw.githubusercontent.com/krishnaik06/Recommendation_complete_tutorial/master/KNN%20Movie%20Recommendation/movies.csv"
movie_df=pd.read_csv(movie_url,usecols=["movieId","title"])

rating_url="https://raw.githubusercontent.com/krishnaik06/Recommendation_complete_tutorial/master/KNN%20Movie%20Recommendation/ratings.csv"

rating_df=pd.read_csv(rating_url,usecols=["userId","movieId","rating"])

In [6]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [8]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [12]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB


In [14]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [16]:
df=pd.merge(rating_df,movie_df,on="movieId")
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [22]:
df.isna().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [27]:
df = df.dropna(axis = 0, subset = ['title'])
totalrating_count=df.groupby(by="title")["rating"].count().reset_index().rename(columns={"rating":"rating_count"})
totalrating_count.head()

,title,rating_count
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [28]:
df_count=df.merge(totalrating_count,left_on="title",right_on="title",how="left")
df_count.head()

,userId,movieId,rating,title,rating_count
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [29]:
df_count["rating_count"].describe()

count    100836.000000
mean         58.758777
std          61.965384
min           1.000000
25%          13.000000
50%          39.000000
75%          84.000000
max         329.000000
Name: rating_count, dtype: float64

In [30]:
popularity_threshold=50
rating_with_popularity=df_count.query("rating_count>=@popularity_threshold")
rating_with_popularity.head()

,userId,movieId,rating,title,rating_count
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [32]:
rating_with_popularity.shape

(41362, 5)

In [34]:
movie_feature_df=rating_with_popularity.pivot_table(index="title",columns="userId",values="rating").fillna(0)
movie_feature_df.head(5)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [35]:
from scipy.sparse import csr_matrix

In [36]:
movie_feature_df_matrix=csr_matrix(movie_feature_df.values)

In [38]:
from sklearn.neighbors import NearestNeighbors

In [42]:
model_knn=NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(movie_feature_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [43]:
movie_feature_df.shape

(450, 606)

In [46]:
movie_index=np.random.choice(movie_feature_df.shape[0])

print(movie_index)

426


In [52]:
distance,indices =model_knn.kneighbors(movie_feature_df.iloc[movie_index,:].values.reshape(1, -1), n_neighbors = 6)

In [54]:
distance.flatten()

array([0.        , 0.41472185, 0.4216671 , 0.4653525 , 0.46543594,
       0.48561232])

In [56]:
for i in range(0,len(distance.flatten())):
  if i==0:
    print(f"Recomendation:{movie_feature_df.index[movie_index]}")
  else:
    print(f"{movie_feature_df.index[indices.flatten()[i]]}, {distance.flatten()[i]}")

Recomendation:Vertigo (1958)
Rear Window (1954), 0.4147218521313296
Chinatown (1974), 0.4216670956627335
Psycho (1960), 0.46535250305836806
Taxi Driver (1976), 0.4654359423748382
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964), 0.48561232476162286
